In [7]:
import os
from datetime import datetime
import requests
import pandas as pd


In [9]:
services = ['Azure Monitor', 'Sentinel']
currencyCodes = ['USD', 'AUD', 'BRL', 'CAD', 'CHF', 'CNY', 'DKK', 'EUR', 'GBP', 'INR', 'JPY', 'KRW', 'NOK', 'NZD', 'RUB', 'SEK', 'TWD']
baseUrl = "https://prices.azure.com/api/retail/prices?api-version=2021-10-01-preview"

previousTier = {
    'Pay-as-you-go' : '',
    '100'  : 'Pay-as-you-go',
    '200'  : 100,
    '300'  : 200,
    '400'  : 300,
    '500'  : 400,
    '1000' : 500,
    '2000' : 1000,
    '5000' : 2000
}

filters = {
    'Azure Monitor' :"$filter=(serviceName eq 'Azure Monitor' or serviceName eq 'Log Analytics') and (meterName eq 'Pay-as-you-go Data Ingestion' or meterName eq '100 GB Commitment Tier Capacity Reservation' or meterName eq '200 GB Commitment Tier Capacity Reservation'  or meterName eq '300 GB Commitment Tier Capacity Reservation' or meterName eq '400 GB Commitment Tier Capacity Reservation' or meterName eq '500 GB Commitment Tier Capacity Reservation' or meterName eq '1000 GB Commitment Tier Capacity Reservation' or meterName eq '2000 GB Commitment Tier Capacity Reservation' or meterName eq '5000 GB Commitment Tier Capacity Reservation')",
    'Sentinel' : "$filter=(serviceName eq 'Sentinel') and (meterName eq 'Pay-as-you-go Analysis' or meterName eq '100 GB Commitment Tier Capacity Reservation' or meterName eq '200 GB Commitment Tier Capacity Reservation' or meterName eq '300 GB Commitment Tier Capacity Reservation' or meterName eq '400 GB Commitment Tier Capacity Reservation' or meterName eq '500 GB Commitment Tier Capacity Reservation' or meterName eq '1000 GB Commitment Tier Capacity Reservation' or meterName eq '2000 GB Commitment Tier Capacity Reservation' or meterName eq '5000 GB Commitment Tier Capacity Reservation')" 
}


In [24]:
class SentinelPricing:
    """This class represents the data from prices.azure.com for specific meters for Azure Monitor and Sentinel."""
    def __init__(self, time_generated, service_name: str, arm_region_name : str, currency_code : str, tier : str, unit_of_measure : str, retail_price : float, effective_start_date : str, effective_next_tier_gb: float = 0.0) -> None:
        self.time_generated = time_generated
        self.service_name = service_name
        self.arm_region_name = arm_region_name
        self.currency_code = currency_code
        self.tier = tier 
        self.unit_of_measure = unit_of_measure 
        self.retail_price = retail_price
        self.effective_start_date = effective_start_date
        self.effective_next_tier_gb = effective_next_tier_gb
        

        if tier == "Pay-as-you-go":
            self.effective_price_per_gb = float(retail_price)
        else:
            self.effective_price_per_gb = float(int(tier) / retail_price)
    
    def add_effective_next_tier_gb(self, previous_tier_effective_price_per_gb):
        self.effective_next_tier_gb = self.retail_price / previous_tier_effective_price_per_gb

    def __str__(self) -> str:
        return "{},{},{},{},{},{},{},{},{}".format(
            self.time_generated,
            self.service_name,
            self.arm_region_name,
            self.currency_code,
            self.tier,
            self.unit_of_measure,
            self.retail_price,
            self.effective_start_date,
            self.effective_next_tier_gb)

In [38]:
def get_sentinel_pricing (service=None, currency_code=None, time_generated=None):

    # Begin: Check Parameters
    if time_generated is None:
        time_generated = datetime.now().strftime('%Y/%m/%d %H:%M:%S')
        print(f'Parameters: service={service}, currencyCode={currency_code}, time_generated=(DefaultParam){time_generated}')
    else:
        print(f'Parameters: service={service}, currencyCode={currency_code}, time_generated={time_generated}')

    if service is None or currency_code is None:
        print(f'You must provide values to the "service" and "currency_code" parameters. Exiting')
        return

    starting_url = f"{baseUrl}&currencyCode={currency_code}&{filters[service]}"
    print(f'\nInitial URL: {starting_url}')

    items_returned = 0
    sentinel_price_list = []

    url = starting_url
    while True:
        response = requests.get(url)
        if response.status_code != 200:
            print("Error: Unable to fetch data")
            break

        data = response.json()
        items_returned += data['Count']

        # Create new instance of the SentinelPricing class and add them to the output list
        for dataItem in data['Items']:
            # Filter out 0 retail prices as we aren't interested in free trials / tiers
            if dataItem['retailPrice'] != 0:
                sentinel_price = SentinelPricing(
                    time_generated = time_generated,
                    service_name = service,
                    arm_region_name = dataItem['armRegionName'],
                    currency_code = currency_code,
                    tier = dataItem['meterName'].split()[0],
                    unit_of_measure = dataItem['unitOfMeasure'],
                    retail_price = dataItem['retailPrice'],
                    effective_start_date = dataItem['effectiveStartDate']
                )

                sentinel_price_list.append(sentinel_price)

        # Determine if there is a next link, if not break the loop
        if data['NextPageLink']:
            url = data['NextPageLink']
        elif data['Count'] == 100:
            url = f'{starting_url}&$skip={items_returned}'
        else:
            break
    
    for x in sentinel_price_list
    
    sentinel_price_list_sorted = sorted(sentinel_price_list, key=lambda x: (x.arm_region_name, x.retail_price))
    
    print(f'Total items_returned: {len(sentinel_price_list)}')
    return sentinel_price_list_sorted


In [26]:
sentinel_price = SentinelPricing(
    time_generated = '13/03/2023 17:46:36',
    service_name = 'Sentinel',
    arm_region_name = 'westus',
    currency_code = 'USD',
    tier = 'Pay-as-you-go',
    unit_of_measure = '1 GB',
    retail_price = '2.6',
    effective_start_date = '24/09/2019 00:00:00')

In [3]:
print(sentinel_price)

13/03/2023 17:46:36,Sentinel,westus,USD,Pay-as-you-go,1 GB,2.6,24/09/2019 00:00:00,0.0


In [37]:
service = "Azure Monitor"
currency_code = "USD"
time_generated = datetime.now().strftime('%Y/%m/%d %H:%M:%S')
sentinel_prices = get_sentinel_pricing(service, currency_code, time_generated)

Parameters: service=Azure Monitor, currencyCode=USD, time_generated=2023/03/14 18:00:33

Initial URL: https://prices.azure.com/api/retail/prices?api-version=2021-10-01-preview&currencyCode=USD&$filter=(serviceName eq 'Azure Monitor' or serviceName eq 'Log Analytics') and (meterName eq 'Pay-as-you-go Data Ingestion' or meterName eq '100 GB Commitment Tier Capacity Reservation' or meterName eq '200 GB Commitment Tier Capacity Reservation'  or meterName eq '300 GB Commitment Tier Capacity Reservation' or meterName eq '400 GB Commitment Tier Capacity Reservation' or meterName eq '500 GB Commitment Tier Capacity Reservation' or meterName eq '1000 GB Commitment Tier Capacity Reservation' or meterName eq '2000 GB Commitment Tier Capacity Reservation' or meterName eq '5000 GB Commitment Tier Capacity Reservation')
Total items_returned: 512


In [39]:
for i in sentinel_prices:
    print(i)

2023/03/14 18:00:33,Azure Monitor,Global,USD,Pay-as-you-go,1 GB,2.3,2018-02-01T00:00:00Z,0.0
2023/03/14 18:00:33,Azure Monitor,attatlanta1,USD,100,1/Day,196.0,2022-04-01T00:00:00Z,0.0
2023/03/14 18:00:33,Azure Monitor,attatlanta1,USD,200,1/Day,368.0,2022-04-01T00:00:00Z,0.0
2023/03/14 18:00:33,Azure Monitor,attatlanta1,USD,300,1/Day,540.0,2022-04-01T00:00:00Z,0.0
2023/03/14 18:00:33,Azure Monitor,attatlanta1,USD,400,1/Day,704.0,2022-04-01T00:00:00Z,0.0
2023/03/14 18:00:33,Azure Monitor,attatlanta1,USD,500,1/Day,865.0,2022-04-01T00:00:00Z,0.0
2023/03/14 18:00:33,Azure Monitor,attatlanta1,USD,1000,1/Day,1700.0,2022-04-01T00:00:00Z,0.0
2023/03/14 18:00:33,Azure Monitor,attatlanta1,USD,2000,1/Day,3320.0,2022-04-01T00:00:00Z,0.0
2023/03/14 18:00:33,Azure Monitor,attatlanta1,USD,5000,1/Day,8050.0,2022-04-01T00:00:00Z,0.0
2023/03/14 18:00:33,Azure Monitor,attdallas1,USD,100,1/Day,219.52,2022-04-01T00:00:00Z,0.0
2023/03/14 18:00:33,Azure Monitor,attdallas1,USD,200,1/Day,412.16,2022-04-01T00:00

In [60]:
for obj in sentinel_prices[0:3]:
    current_tier = obj.tier
    previous_tier = previousTier.get(current_tier)

    print("currentTierType = {}, currentTierValue = {}".format(type(current_tier),current_tier))
    print("previousTierType = {}, previousTierValue = {}".format(type(previous_tier),previous_tier))
    
    armRegionTest = [x for x in sentinel_prices if x.arm_region_name == obj.arm_region_name]
    tierTest = [x for x in sentinel_prices if x.tier == previous_tier]
    print("ARMTest: {}".format(armRegionTest))
    print("TierTest: {}".format(tierTest))

    #previous_tier_obj = [x for x in sentinel_prices if x.arm_region_name == obj.arm_region_name and x.tier == previous_tier]

    #print(previous_tier_obj)
    #if len(previous_tier_obj) > 1:
    #    print("Warning: Multiple objects returned when determining previous tier")

    #if len(previous_tier_obj) > 0:
    #    previous_tier_obj = previous_tier_obj[0]
    #    obj.add_effective_next_tier_gb(previous_tier_obj.effective_price_per_gb)
    #    print(obj.effective_next_tier_gb)
    #else:
    #    print(f"Length = {len(previous_tier_obj)}")
        
    

currentTierType = <class 'str'>, currentTierValue = Pay-as-you-go
previousTierType = <class 'str'>, previousTierValue = 
ARMTest: [<__main__.SentinelPricing object at 0x0000016EB2B29A00>]
TierTest: []
currentTierType = <class 'str'>, currentTierValue = 100
previousTierType = <class 'str'>, previousTierValue = Pay-as-you-go
ARMTest: [<__main__.SentinelPricing object at 0x0000016EB2C57280>, <__main__.SentinelPricing object at 0x0000016EB2C68B20>, <__main__.SentinelPricing object at 0x0000016EB2BB8F10>, <__main__.SentinelPricing object at 0x0000016EB2C68910>, <__main__.SentinelPricing object at 0x0000016EB2B29970>, <__main__.SentinelPricing object at 0x0000016EB2B24EE0>, <__main__.SentinelPricing object at 0x0000016EB2B014C0>, <__main__.SentinelPricing object at 0x0000016EB2B78160>]
TierTest: [<__main__.SentinelPricing object at 0x0000016EB2B29A00>, <__main__.SentinelPricing object at 0x0000016EB2BB89D0>, <__main__.SentinelPricing object at 0x0000016EB2C281F0>, <__main__.SentinelPricing o